<h1>Quantum SoundWave</h1>
<h4>By Kevin Li (<a href=mailto:bluekevin2003@gmail.com>bluekevin2003@gmail.com</a>, <a href=mailto:kjli@wisc.edu>kjli@wisc.edu</a>)</h4>

In [ ]:
# Name: Kevin Li
# School Email: kjli@wisc.edu
# Personal Email: bluekevin2003@gmail.com
# Last Updated: 10/23/2023
# Version Number: 1.0.0

Welcome to Quantum SoundWave. The intent of this program will be to use the properties of superpositioning and entanglement in qubits to produce sets of sound waves with various interesting properties. To start, we will import the standard Qiskit libraries as well as helpful modules.

In [ ]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile
from qiskit.circuit import QuantumRegister
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit_aer import AerSimulator

In [ ]:
# Importing general modules/libraries
import math # https://docs.python.org/3/library/math.html
import numpy as np # https://numpy.org/
import pyaudio # https://people.csail.mit.edu/hubert/pyaudio/

First, let us familiarize ourselves with sound waves. Let's create an example.

In [ ]:
"""Base code from https://stackoverflow.com/questions/8299303/generating-sine-wave-sound-in-python
and https://stackoverflow.com/questions/9770073/sound-generation-synthesis-with-python"""

# Initialize PyAudio stream
frequency = 440.0 # 440 Hz (A4)
sampling_rate = math.ceil(frequency * 100 + 100) # Frames per second where function is checked
p = pyaudio.PyAudio()
stream = p.open(
    format=pyaudio.paFloat32,
    rate=sampling_rate,
    channels=1, # Number of channels from which sound can come from
    output=True,
)

# Construct the sine wave
amplitude = 0.5 # volume, range [0.0, 1.0]
duration = 2.0 # 2 seconds
wave = (amplitude * (np.sin(2 * np.pi * frequency * np.arange(duration * sampling_rate) / sampling_rate).astype(np.float32))).tobytes()


# Play the sound
stream.write(wave)

# Close stream, terminate PyAudio to prevent memory leaks
stream.stop_stream()
stream.close()
p.terminate()

We can change the frequency variable to change the pitch, the amplitude variable to change the volume, and the duration variable to change the duration the sound wave is played. That's neat, what if we want to play multiple waves? We can add the wave functions together.

Let's create a function for this.

In [ ]:

def playSound(count: int, frequencies: float | list[float], amplitudes: float | list[float], duration: float):
    """
    Play the sound represented by one or more sine waves.
    It is assumed that the length of the frequencies and amplitudes arguments equal to count.

    :param count: number of wave functions
    :param frequencies: frequency (Hz) of each wave function
    :param amplitudes: amplitude [0.0, 1.0] of each wave function
    :param duration: the time length to play the sound
    """

    # Convert data types of frequencies and amplitudes to lists if they are single floats
    if type(frequencies) == float:
        frequencies = [frequencies]
    if type(amplitudes) == float:
        amplitudes = [amplitudes]

    # Initialize PyAudio stream
    sampling_rate = math.ceil(max(frequencies) * 100 + 100) # Frames per second where function is checked
    p = pyaudio.PyAudio()
    stream = p.open(
        format=pyaudio.paFloat32,
        rate=sampling_rate,
        channels=1,
        output=True,
    )

    # Construct the sine wave
    wave = np.zeros(int(duration * sampling_rate), dtype=np.float32)
    for i in range(count):
        wave = wave + amplitudes[i] * (np.sin(2 * np.pi * frequencies[i] * np.arange(duration * sampling_rate) / sampling_rate).astype(np.float32))

    # Play the sound
    stream.write(wave.tobytes())

    # Close stream, terminate PyAudio to prevent memory leaks
    stream.stop_stream()
    stream.close()
    p.terminate()

Now let's test it out!

In [ ]:
playSound(2, (440.0, 660.0), (0.8, 0.2), 2.0) # A4, E5

Warning: Make sure that the sum of the amplitudes don't exceed 1.0. Otherwise, you'll get something like this:

In [ ]:
playSound(2, (440.0, 660.0), (0.8, 0.3), 2.0)

Let's try incorporating qubits into this. Maybe we want the rotation of a qubit to determine the amplitude of the sound wave; for example, we may want to hear the sound the loudest if the qubit is in a $\left|1\right>$ state, but to not hear it at all if it is in a $\left|0\right>$ state. More precisely, if a qubit is in a state of $\left|\Psi\right>$, we want an amplitude of 0.0 when $|\Psi_1|^2=0$, 1.0 when $|\Psi_1|^2=1$, and anything in between. Let's construct a quantum circuit:

In [ ]:
qc = QuantumCircuit(3, 3) # q0 will be silent, q1 will be loud, q2 will be in between
qc.x(1) # Flip q1 to |1>
qc.h(2) # Flip q2 to |+> = sqrt(1/2)|0> + sqrt(1/2)|1>
qc.measure([0, 1, 2], [0, 1, 2])

Since measuring a qubit collapses the state into one of the standard basis states, we can determine the desired amplitude by measuring many qubits passing through the circuit.

In [ ]:
shots = 1000 # Number of measurements per qubit, increase for greater accuracy

sim = AerSimulator()
job = sim.run(qc, shots=shots)
result = job.result()
counts = result.get_counts()

print(counts)

Now let's divide each of the qubit measurement counts into the desired amplitudes (remember that Qiskit uses little-endian notation for qubits). We can create a function and invoke it for any set of measurements.

In [ ]:
def qubitsToAmplitudes(counts: dict[str, int], numQubitRegisters: int, shots: int) -> list[float]:
    """
    Determine the amplitude of each sound wave based on its respective qubit measurement count.

    :param numQubitRegisters: the number of qubit registers in the circuit
    :param counts: the dictionary of measurements
    :param shots: the number of shots used for the measurement counts
    :return the list of amplitudes
    """
    amps = [0.0] * numQubitRegisters
    for i in counts:
        for j in range(numQubitRegisters):
            if int(i[-j - 1]) == 1:
                amps[j] += counts[i]
    for i in range(numQubitRegisters):
        amps[i] /= shots
    return amps

amps = qubitsToAmplitudes(counts=counts, numQubitRegisters=3, shots=shots)
print(amps)

This will only be an approximation, but one can increase the accuracy by running more shots in the simulation measurement.

Now let's play the respective sound waves:

In [ ]:
playSound(1, 440.0, amps[0], 2.0) # q0, zero amplitude
playSound(1, 440.0, amps[1], 2.0) # q1, full amplitude
playSound(1, 440.0, amps[2], 2.0) # q2, about half amplitude

We can do this for any qubit state. Here's a function for adding various gates to a quantum register. Feel free to download this notebook to add, remove, or change the gates as desired.

In [ ]:
def addVariousGates(qc: QuantumCircuit, qr: QuantumRegister):
    # TODO: add, remove, and change gates below as desired
    qc.h(qr)
    qc.x(qr)
    qc.sdg(qr)
    qc.x(qr)
    qc.h(qr)
    qc.x(qr)
    qc.p(0.69, qr)
    qc.y(qr)
    qc.h(qr)

This is nice, but what else can we do with qubits?

Suppose we want to play an interval or chord with matching volumes. We can use the entanglement property of qubits to our advantage.

In [ ]:
qc = QuantumCircuit(3, 3)
addVariousGates(qc, qc.qubits[0]) # Add gates to register q0
qc.cx(0, 1) # Entangle q0 and q1
qc.cx(0, 2) # Entangle q0 and q2
qc.measure([0, 1, 2], [0, 1, 2])

job = sim.run(qc, shots=shots)
result = job.result()
counts = result.get_counts()

print(counts)

The amplitudes will be the same:

In [ ]:
amps = qubitsToAmplitudes(counts=counts, numQubitRegisters=3, shots=shots)
print(amps)

In [ ]:
playSound(3, [440.0, 523.25, 666.0], amps, 2.0) # a minor chord

Side note: You can also have negative amplitudes for your sound waves:

In [ ]:
playSound(2, (440.0, 660.0), (0.4, -0.5), 2.0)

Then, if you were to put two sound waves with the same frequency together, they would interfere with each other either constructively or destructively, depending on their amplitudes.

In [ ]:
playSound(2, (440.0, 440.0), (0.5, 0.5), 2.0) # Constructive, magnitude of total amplitude = 1.0
playSound(2, (440.0, 440.0), (0.5, -0.5), 2.0) # Destructive, magnitude of total amplitude = 0.0 (silent)
playSound(2, (440.0, 440.0), (-0.5, -0.5), 2.0) # Constructive, magnitude of total amplitude = 1.0

Now, let's try this with a quantum system.

We'll need to update qubitsToAmplitudes. This time, we want the amplitude to equal to $|\Psi_1|^2-|\Psi_0|^2$. Thus, for each qubit measured, we want to add to the amplitude when a 1 is measured, and subtract to the amplitude when a 0 is measured.

In [ ]:
def qubitsToAmplitudesUpdated(counts: dict[str, int], numQubitRegisters: int, shots: int) -> list[float]:
    """
    Determine the amplitude of each sound wave based on its respective qubit measurement count.
    Negative amplitudes are included.

    :param numQubitRegisters: the number of qubit registers in the circuit
    :param counts: the dictionary of measurements
    :param shots: the number of shots used for the measurement counts
    :return the list of amplitudes
    """
    amps = [0.0] * numQubitRegisters
    for i in counts:
        for j in range(numQubitRegisters):
            if int(i[-j - 1]) == 1:
                amps[j] += counts[i]
            else:
                amps[j] -= counts[i]
    for i in range(numQubitRegisters):
        amps[i] /= shots
    return amps

Here's constructive interference...

In [ ]:
qc = QuantumCircuit(2, 2)
qc.ry(2.0, 0) # Sample gate: ensure q0 amplitude will be sufficiently positive but not exceed 0.5
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])

job = sim.run(qc, shots=shots)
result = job.result()
counts = result.get_counts()

amps = qubitsToAmplitudesUpdated(counts=counts, numQubitRegisters=2, shots=shots)
if (amps[0] + amps[1] > 1.0):
    # Prevent total amplitude from exceeding 1.0 in edge cases
    diff = (amps[0] + amps[1] - 1.0) / 2.0
    amps[0] -= diff
    amps[1] -= diff
print(amps)

# The soundwave by itself
playSound(1, 440.0, amps[0], 2.0)

# These two should be equivalent
playSound(2, [440.0, 440.0], amps, 2.0)
playSound(1, 440.0, sum(amps), 2.0)

and destructive interference...

In [ ]:
qc = QuantumCircuit(2, 2)
qc.ry(2.0, 0)
qc.x(1) # NOT gate on q1
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])

job = sim.run(qc, shots=shots)
result = job.result()
counts = result.get_counts()

amps = qubitsToAmplitudesUpdated(counts=counts, numQubitRegisters=2, shots=shots)
print(amps)

# The soundwave by itself
playSound(1, 440.0, amps[0], 2.0)

# These two should be equivalent (and silent)
playSound(2, [440.0, 440.0], amps, 2.0)
playSound(1, 440.0, sum(amps), 2.0)

Ideally, we would be able to use exploit more properties of qubits to sound waves, such as continuously and dynamically changing the amplitudes as we change the quantum circuit or make new measurements on qubit streams, or having the qubits determine the frequencies instead.

Unfortunately, due to limitations in Qiskit, Python, and time, this project will not delve into those prospects into depth. Regardless, I hope you enjoyed this exploration into connecting qubits and sound waves. Thank you for taking the time to read, listen, and run this! I hope you have a great day.

Bonus: try running the below code a few times. (Note: since the quantum circuit produced is random, it will sometimes produce an error message when finding the running the circuit and obtaining the results.) What sounds did you get?

In [ ]:
from qiskit.circuit.random import random_circuit

def qubitsToFrequencies(counts: dict[str, int], numQubitRegisters: int, shots: int) -> list[float]:
    """
    Determine the frequency of each sound wave based on its respective qubit measurement count.

    :param numQubitRegisters: the number of qubit registers in the circuit
    :param counts: the dictionary of measurements
    :param shots: the number of shots used for the measurement counts
    :return the list of frequency
    """
    freqs = [0.0] * numQubitRegisters
    for i in counts:
        for j in range(numQubitRegisters):
            if int(i[-j - 1]) == 1:
                freqs[j] += counts[i]
    for i in range(numQubitRegisters):
        freqs[i] = freqs[i] * 440.0 / shots + 440.0 # All results will be between A4 and A5, inclusive
    return freqs

qc = random_circuit(2, 3, measure=True, max_operands=1)
job = sim.run(qc, shots=shots)
result = job.result()
counts = result.get_counts()
freqs = qubitsToFrequencies(counts=counts, numQubitRegisters=2, shots=shots)
print(freqs)
playSound(2, freqs, [0.3, 0.3], 2.0)